In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, InputLayer, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger

import warnings
#warnings.filterwarnings("ignore")
#tf version should be 2.9.0
print(tf.__version__)

2.9.0


In [2]:
rand_seed = 42

In [3]:
dir_path = r"C:\Users\acer\Desktop\Data_2D\data_x"
results_dir = os.path.join(dir_path, r'logs\EfficientNet-B5')
models_dir = os.path.join(dir_path, r'models\EfficientNet-B5')

if not os.path.isdir(results_dir):
    os.makedirs(results_dir)
    
if not os.path.isdir(models_dir):
    os.makedirs(models_dir)

In [4]:
# define the location of your dataset

TRAIN_PATH = r"C:\Users\acer\Desktop\Data_2D\data_x\train"
TRAIN_LABEL_PATH = r"C:\Users\acer\Desktop\Data_2D\data_x\train_classification.csv"

VAL_PATH = r"C:\Users\acer\Desktop\Data_2D\data_x\validation"
VAL_LABEL_PATH = r"C:\Users\acer\Desktop\Data_2D\data_x\validation_classification.csv"

IMG_DIM = (224,224)
INPUT_SHAPE = (224,224,3)

BATCH_SIZE = 32
EPOCH = 20
x_axis_inc = 1 # for plotting the training acc and loss

In [5]:
import pandas as pd

def expand_dataframe(df):
    # Create a list to hold the new rows
    new_rows = []
    
    # Iterate through each row in the original dataframe
    for _, row in df.iterrows():
        # Duplicate the row 234 times (for indices 0-233)
        for i in range(224):
            new_row = row.copy()
            new_row['ID'] = f"{row['ID']}_x_{i:03d}"  # Wrap the original ID and add suffix
            new_rows.append(new_row)
    
    # Create a new dataframe from the list of new rows
    new_df = pd.DataFrame(new_rows)
    
    return new_df

train_original_df = pd.read_csv(TRAIN_LABEL_PATH)
train_c_erosion_label = train_original_df[['ID','c_erosion']]
# Create the new DataFrame
train_df = expand_dataframe(train_c_erosion_label)

# Shuffle the training data
train_df = train_df.sample(frac=1, random_state=rand_seed).reset_index(drop=True)

train_df.head()

,ID,c_erosion
0,57-10397 R 2015_x_100,0
1,47-4881 R 2018_x_156,1
2,66-2177 L_x_216,0
3,61-20310 R_x_089,1
4,54-50 L_x_059,0


In [6]:
data_labels = train_df
target_labels = data_labels['c_erosion']
data_labels['image_path'] =  data_labels.apply(lambda row: (os.path.join(TRAIN_PATH, str(row['ID'])) + '.jpg'), axis=1)
data_labels.head()

,ID,c_erosion,image_path
0,57-10397 R 2015_x_100,0,C:\Users\acer\Desktop\Data_2D\data_x\train\57-...
1,47-4881 R 2018_x_156,1,C:\Users\acer\Desktop\Data_2D\data_x\train\47-...
2,66-2177 L_x_216,0,C:\Users\acer\Desktop\Data_2D\data_x\train\66-...
3,61-20310 R_x_089,1,C:\Users\acer\Desktop\Data_2D\data_x\train\61-...
4,54-50 L_x_059,0,C:\Users\acer\Desktop\Data_2D\data_x\train\54-...


In [7]:
print(data_labels.shape)

(56896, 3)


In [8]:
import pandas as pd
import os

def check_valid_files(df, column_name='image_path'):
    # Create a new column for validity
    df['is_valid_file'] = df[column_name].apply(os.path.isfile)
    
    # Print summary
    total = len(df)
    valid = df['is_valid_file'].sum()
    invalid = total - valid
    
    print(f"Total files: {total}")
    print(f"Valid files: {valid}")
    print(f"Invalid files: {invalid}")
    
    # If there are invalid files, you can get them like this:
    if invalid > 0:
        print("\nInvalid files:")
        invalid_files = df[~df['is_valid_file']][column_name]
        for file in invalid_files:
            print(file)
    
    return df

check_valid_file_df = check_valid_files(data_labels)

# You can access the results in the DataFrame
check_valid_file_df.head(10)

Total files: 56896
Valid files: 56896
Invalid files: 0


,ID,c_erosion,image_path,is_valid_file
0,57-10397 R 2015_x_100,0,C:\Users\acer\Desktop\Data_2D\data_x\train\57-...,True
1,47-4881 R 2018_x_156,1,C:\Users\acer\Desktop\Data_2D\data_x\train\47-...,True
2,66-2177 L_x_216,0,C:\Users\acer\Desktop\Data_2D\data_x\train\66-...,True
3,61-20310 R_x_089,1,C:\Users\acer\Desktop\Data_2D\data_x\train\61-...,True
4,54-50 L_x_059,0,C:\Users\acer\Desktop\Data_2D\data_x\train\54-...,True
5,60-7166 L_x_149,0,C:\Users\acer\Desktop\Data_2D\data_x\train\60-...,True
6,61-700847 R_x_104,0,C:\Users\acer\Desktop\Data_2D\data_x\train\61-...,True
7,65-27889 L_x_222,0,C:\Users\acer\Desktop\Data_2D\data_x\train\65-...,True
8,61-20310 R_x_121,1,C:\Users\acer\Desktop\Data_2D\data_x\train\61-...,True
9,64-8619 L 2022_x_051,0,C:\Users\acer\Desktop\Data_2D\data_x\train\64-...,True


In [9]:
val_original_df = pd.read_csv(VAL_LABEL_PATH)
val_c_erosion_label = val_original_df[['ID','c_erosion']]
# Create the new DataFrame
val_df = expand_dataframe(val_c_erosion_label)

# Shuffle the validation data
val_df = val_df.sample(frac=1, random_state=rand_seed).reset_index(drop=True)

val_df.head()

,ID,c_erosion
0,63-17877 R_x_020,0
1,61-5586 R_x_196,0
2,63-17877 R_x_090,0
3,60-34373 L 2019_x_113,0
4,53-24700 R_x_092,0


In [10]:
val_labels = val_df[['ID','c_erosion']]
target_val_labels = val_labels['c_erosion']
val_labels['image_path'] =  val_labels.apply(lambda row: (os.path.join(VAL_PATH, str(row['ID'])) + '.jpg'), axis=1)
val_labels.head()

,ID,c_erosion,image_path
0,63-17877 R_x_020,0,C:\Users\acer\Desktop\Data_2D\data_x\validatio...
1,61-5586 R_x_196,0,C:\Users\acer\Desktop\Data_2D\data_x\validatio...
2,63-17877 R_x_090,0,C:\Users\acer\Desktop\Data_2D\data_x\validatio...
3,60-34373 L 2019_x_113,0,C:\Users\acer\Desktop\Data_2D\data_x\validatio...
4,53-24700 R_x_092,0,C:\Users\acer\Desktop\Data_2D\data_x\validatio...


In [11]:
check_valid_file_df = check_valid_files(val_labels)
check_valid_file_df.head()

Total files: 12096
Valid files: 12096
Invalid files: 0


,ID,c_erosion,image_path,is_valid_file
0,63-17877 R_x_020,0,C:\Users\acer\Desktop\Data_2D\data_x\validatio...,True
1,61-5586 R_x_196,0,C:\Users\acer\Desktop\Data_2D\data_x\validatio...,True
2,63-17877 R_x_090,0,C:\Users\acer\Desktop\Data_2D\data_x\validatio...,True
3,60-34373 L 2019_x_113,0,C:\Users\acer\Desktop\Data_2D\data_x\validatio...,True
4,53-24700 R_x_092,0,C:\Users\acer\Desktop\Data_2D\data_x\validatio...,True


In [12]:
train_data_size = 1000
validation_data_size = 200

In [13]:
train_data = np.array([img_to_array(load_img(img, target_size=IMG_DIM))
                       for img in data_labels['image_path'][0:train_data_size].values.tolist()]).astype('float32')

In [14]:
val_data = np.array([img_to_array(load_img(img, target_size=IMG_DIM))
                       for img in val_labels['image_path'][0:validation_data_size].values.tolist()]).astype('float32')

In [15]:
print('Training Dataset Size:', train_data.shape)
print('Validation Dataset Size:', val_data.shape)

Training Dataset Size: (1000, 224, 224, 3)
Validation Dataset Size: (200, 224, 224, 3)


## Prepare for Transfer Learning

In [16]:
prep_in = tf.keras.applications.efficientnet.preprocess_input

In [17]:
x_train_new = train_data.astype('int')
x_val_new = val_data.astype('int')

In [18]:
x_train_in = prep_in(x_train_new)
x_val_in = prep_in(x_val_new)

In [19]:
train_labels_enc = target_labels[0:train_data_size].to_numpy()
val_labels_enc = target_val_labels[0:validation_data_size].to_numpy()

In [22]:
# Get the VGG16 model so we can do transfer learning
base_model = tf.keras.applications.EfficientNetB5(input_shape=INPUT_SHAPE, include_top=False, weights='imagenet')

In [23]:
print('Number of layers in the base model: ', len(base_model.layers))

Number of layers in the base model:  577


In [24]:
pd.set_option('max_colwidth', None)
layers = [(layer, layer.name, layer.trainable) for layer in base_model.layers]
pd.DataFrame(layers, columns=['Layer Type', 'Layer Name', 'Layer Trainable'])

,Layer Type,Layer Name,Layer Trainable
0,<keras.engine.input_layer.InputLayer object at 0x00000172D9D534F0>,input_1,True
1,<keras.layers.preprocessing.image_preprocessing.Rescaling object at 0x00000172DC837760>,rescaling,True
2,<keras.layers.preprocessing.normalization.Normalization object at 0x00000172DC837310>,normalization,True
3,<keras.layers.core.tf_op_layer.TFOpLambda object at 0x00000172DF756040>,tf.math.truediv,True
4,<keras.layers.reshaping.zero_padding2d.ZeroPadding2D object at 0x00000172DC837D30>,stem_conv_pad,True
...,...,...,...
572,<keras.layers.regularization.dropout.Dropout object at 0x00000172DABCF550>,block7c_drop,True
573,<keras.layers.merging.add.Add object at 0x00000172DABD98B0>,block7c_add,True
574,<keras.layers.convolutional.conv2d.Conv2D object at 0x00000172D9E6CD60>,top_conv,True
575,<keras.layers.normalization.batch_normalization.BatchNormalization object at 0x00000172DE0BDB50>,top_bn,True


## Slapping AvgPool > 1024_Dense > 512_Dense > 1_output on top of the EfficientNet

In [25]:
base_model.trainable = False

pd.set_option('max_colwidth', None)
layers = [(layer, layer.name, layer.trainable) for layer in base_model.layers[10:]]
pd.DataFrame(layers, columns=['Layer Type', 'Layer Name', 'Layer Trainable'])

,Layer Type,Layer Name,Layer Trainable
0,<keras.layers.core.activation.Activation object at 0x00000172D819AB50>,block1a_activation,False
1,<keras.layers.pooling.global_average_pooling2d.GlobalAveragePooling2D object at 0x00000172DF756AF0>,block1a_se_squeeze,False
2,<keras.layers.reshaping.reshape.Reshape object at 0x00000172DA0A5FA0>,block1a_se_reshape,False
3,<keras.layers.convolutional.conv2d.Conv2D object at 0x00000172DA0A5A60>,block1a_se_reduce,False
4,<keras.layers.convolutional.conv2d.Conv2D object at 0x00000172DA21FF10>,block1a_se_expand,False
...,...,...,...
562,<keras.layers.regularization.dropout.Dropout object at 0x00000172DABCF550>,block7c_drop,False
563,<keras.layers.merging.add.Add object at 0x00000172DABD98B0>,block7c_add,False
564,<keras.layers.convolutional.conv2d.Conv2D object at 0x00000172D9E6CD60>,top_conv,False
565,<keras.layers.normalization.batch_normalization.BatchNormalization object at 0x00000172DE0BDB50>,top_bn,False


In [26]:
fig_train = 'EfficientNetB5_A_FE_train_224'
log_file = os.path.join(results_dir, 'EfficientNetB5_A_FE_train_224.csv')
model_path = os.path.join(models_dir, 'EfficientNetB5_A_FE_224.h5')

In [27]:
# Create a new model on top

inputs = tf.keras.Input(shape=INPUT_SHAPE)
# We make sure that the base_model is running in inference mode here,
# by passing 'training=False'. This is important for fine-tuning
x = base_model(inputs, training=False)

x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dense(512, activation='relu')(x)

# A Dense classifier with a single unit (binary classification)
outputs = Dense(1, activation='sigmoid')(x)
model = Model(inputs, outputs)

In [28]:
pd.set_option('max_colwidth', None)
layers = [(layer, layer.name, layer.trainable) for layer in model.layers]
pd.DataFrame(layers, columns=['Layer Type', 'Layer Name', 'Layer Trainable'])

,Layer Type,Layer Name,Layer Trainable
0,<keras.engine.input_layer.InputLayer object at 0x00000172DAD2FCA0>,input_2,True
1,<keras.engine.functional.Functional object at 0x00000172DABD2DC0>,efficientnetb5,False
2,<keras.layers.pooling.global_average_pooling2d.GlobalAveragePooling2D object at 0x00000172DAD2FB80>,global_average_pooling2d,True
3,<keras.layers.core.dense.Dense object at 0x00000172DD2E1640>,dense,True
4,<keras.layers.core.dense.Dense object at 0x00000172DE22CFD0>,dense_1,True
5,<keras.layers.core.dense.Dense object at 0x00000172DAD2FD00>,dense_2,True


In [29]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 efficientnetb5 (Functional)  (None, 7, 7, 2048)       28513527  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 1024)              2098176   
                                                                 
 dense_1 (Dense)             (None, 512)               524800    
                                                                 
 dense_2 (Dense)             (None, 1)                 513       
                                                             

In [30]:
callbacks = [
    CSVLogger(log_file),
    ModelCheckpoint(
        filepath=os.path.join(models_dir,"Eff-epoch-{epoch:02d}.h5"),
        save_weights_only=False,
        save_best_only=False,
        save_freq='epoch',
        verbose=1
    )
]

In [31]:
model.compile(loss=keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer= keras.optimizers.RMSprop(lr=1e-3),
              metrics = [keras.metrics.BinaryAccuracy()])

c:\Users\acer\anaconda3\envs\example2D\lib\site-packages\keras\optimizers\optimizer_v2\rmsprop.py:135: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [32]:
history = model.fit(x=x_train_in, 
                    y=train_labels_enc,
                    validation_data=(x_val_in, val_labels_enc),
                    epochs=EPOCH, 
                    verbose=1, 
                    callbacks=callbacks)

Epoch 1/20


c:\Users\acer\anaconda3\envs\example2D\lib\site-packages\tensorflow\python\util\dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


32/32 [==============================] - ETA: 0s - loss: 0.7849 - binary_accuracy: 0.5730
Epoch 1: saving model to C:\Users\acer\Desktop\Data_2D\data_x\models\EfficientNet-B5\Eff-epoch-01.h5
32/32 [==============================] - 103s 3s/step - loss: 0.7849 - binary_accuracy: 0.5730 - val_loss: 0.6571 - val_binary_accuracy: 0.6450
Epoch 2/20
32/32 [==============================] - ETA: 0s - loss: 0.6446 - binary_accuracy: 0.6350
Epoch 2: saving model to C:\Users\acer\Desktop\Data_2D\data_x\models\EfficientNet-B5\Eff-epoch-02.h5
32/32 [==============================] - 90s 3s/step - loss: 0.6446 - binary_accuracy: 0.6350 - val_loss: 0.6716 - val_binary_accuracy: 0.5700
Epoch 3/20
32/32 [==============================] - ETA: 0s - loss: 0.5975 - binary_accuracy: 0.6500
Epoch 3: saving model to C:\Users\acer\Desktop\Data_2D\data_x\models\EfficientNet-B5\Eff-epoch-03.h5
32/32 [==============================] - 89s 3s/step - loss: 0.5975 - binary_accuracy: 0.6500 - val_loss: 0.6582 - val

In [33]:
history_dict = history.history
history_dict.keys()

dict_keys(['loss', 'binary_accuracy', 'val_loss', 'val_binary_accuracy'])

In [34]:
epoch_list = list(range(1,EPOCH+1))
y_train_acc = history.history['binary_accuracy']
y_train_loss = history.history['loss']

In [35]:
print(y_train_acc)

[0.5730000138282776, 0.6349999904632568, 0.6499999761581421, 0.6930000185966492, 0.703000009059906, 0.7319999933242798, 0.7490000128746033, 0.7540000081062317, 0.7599999904632568, 0.7910000085830688, 0.7929999828338623, 0.7940000295639038, 0.8019999861717224, 0.8019999861717224, 0.828000009059906, 0.8050000071525574, 0.824999988079071, 0.8209999799728394, 0.8309999704360962, 0.824999988079071]


In [36]:
print(y_train_loss)

[0.7848941683769226, 0.6446096897125244, 0.5975244045257568, 0.5425624847412109, 0.5307867527008057, 0.47771456837654114, 0.46382439136505127, 0.4536462128162384, 0.4142265021800995, 0.3968580663204193, 0.3868872821331024, 0.37695661187171936, 0.3520560562610626, 0.3339458405971527, 0.3312993049621582, 0.3376658260822296, 0.2826150953769684, 0.2989995777606964, 0.31528350710868835, 0.2996891140937805]
